In [5]:
import sys
sys.path.insert(0, "../")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import cv2
from pathlib import Path
import tifffile
import gc
from src.utils.utils import rle2mask, IMAGE_SIZES, mask2rle
import rasterio

In [6]:
INPUT_PATH = Path("/hdd/kaggle/hubmap/input_v2")   
df_train = pd.read_csv(INPUT_PATH / "train.csv")
train_info = pd.read_csv(INPUT_PATH / "HuBMAP-20-dataset_information.csv")
train_tiffs = list((INPUT_PATH / "train").glob('*.tiff'))

In [7]:
def get_tiff(img_id, path="/hdd/kaggle/hubmap/input_v2/test"):
    img_path = Path(path) / f"{img_id}.tiff"
    dataset = rasterio.open(img_path, num_threads="all_cpus")
    # mask_main, mask_second = create_masks(img_id, rle, path)
    h, w = dataset.height, dataset.width
    return h, w

In [4]:
df_sub = pd.read_csv("/hdd/kaggle/hubmap/input_v2/sample_submission.csv").set_index("id", drop=True)

In [9]:
!ls -l ./public_predicts/

total 55612
-rw-rw-r-- 1 trytolose trytolose 6411831 Apr 11 11:18 1024_256_score_0.9166_lb_0.923.csv
-rw-rw-r-- 1 trytolose trytolose 6372875 Apr 12 11:17 924lb.csv
-rw-rw-r-- 1 trytolose trytolose     141 Apr 14 13:00 dataset-metadata.json
-rw-rw-r-- 1 trytolose trytolose 6961344 Apr 11 21:51 effnet-0.csv
-rw-rw-r-- 1 trytolose trytolose 6378633 Apr 11 22:20 effnet-1.csv
-rw-rw-r-- 1 trytolose trytolose 6256218 Apr 11 19:44 hard_augs.csv
-rw-rw-r-- 1 trytolose trytolose 5586532 Apr 12 23:25 no_resize_1024_lb_920.csv
-rw-rw-r-- 1 trytolose trytolose 6314221 Apr 11 13:49 public_aug_w0.5_epoch_34_score_0.9175_lb_0.921.csv
-rw-rw-r-- 1 trytolose trytolose 6355181 Apr 11 11:19 public_aug_w0.5_myloss_epoch_34_score_0.9175_lb_0.918.csv
-rw-rw-r-- 1 trytolose trytolose 6288705 Apr 11 20:01 random_4096_1024_lb_921.csv


In [5]:
df_no_resize = pd.read_csv("./public_predicts/no_resize_1024_lb_920.csv").set_index("id", drop=True)

In [20]:
img_id = '2ec3f1bb9'
mask_rle = df_no_resize.loc[img_id, 'predicted']
h, w = get_tiff(img_id)
mask = rle2mask(mask_rle, (w, h))

/home/trytolose/miniconda3/envs/audio/lib/python3.8/site-packages/rasterio/__init__.py:207: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


In [21]:
nb_components, output, stats, centroids = cv2.connectedComponentsWithStats(mask, connectivity=8)
sizes = stats[:, -1]

In [6]:
def filter_image(img_id, rle, min_sq=1000):
    
    path="/hdd/kaggle/hubmap/input_v2/test"
    img_path = Path(path) / f"{img_id}.tiff"
    dataset = rasterio.open(img_path, num_threads="all_cpus")
    h, w = dataset.height, dataset.width
    mask = rle2mask(rle, (w, h))

    nb_components, output, stats, centroids = cv2.connectedComponentsWithStats(mask, connectivity=8)
    sizes = stats[:, -1]
    for i in range(1, nb_components):
        if sizes[i] < min_sq:
            mask[output == i] = 0
    
    rle_out = mask2rle(mask)
    del mask, dataset, nb_components, output, stats, centroids
    gc.collect()
    gc.collect()
    return rle_out

In [ ]:
for img_id, row in tqdm(df_no_resize.iterrows(), total=5):
    df_sub.loc[img_id, 'predicted'] = filter_image(img_id, row['predicted'])

  0%|          | 0/5 [00:00<?, ?it/s]/home/trytolose/miniconda3/envs/audio/lib/python3.8/site-packages/rasterio/__init__.py:207: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
 40%|████      | 2/5 [01:12<01:45, 35.05s/it]

In [ ]:
df_sub = df_sub.reset_index()
df_sub.to_csv("1024_no_resize_filtered.csv", index=False)

In [4]:
!ls -lh ../../input/train_1024_512

total 2.1M
drwxrwxr-x 2 trytolose trytolose 496K May  3 22:37 images
drwxrwxr-x 2 trytolose trytolose 496K May  3 22:37 masks
-rw-rw-r-- 1 trytolose trytolose 1.1M May  3 22:37 meta.csv


In [74]:
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
import numpy as np

# X = np.array([[1,2], [3,4], [1,2], [3,4], [1,2], [3,4], [1,2], [3,4]])
# y = np.array([[0,0], [0,0], [0,1], [0,1], [1,1], [1,1], [1,0], [1,0]])

# mskf = MultilabelStratifiedKFold(n_splits=2, shuffle=True, random_state=0)

# for train_index, test_index in mskf.split(X, y):
#    print("TRAIN:", train_index, "TEST:", test_index)
#    X_train, X_test = X[train_index], X[test_index]
#    y_train, y_test = y[train_index], y[test_index]

In [101]:
df = pd.read_csv("../../input/train_1024_512/meta.csv")
strf_cols = ['glomerulus_pix','medulla','cortex','outer_stripe','Inner medulla','Outer Medulla']
for col in strf_cols:
    df[col] = pd.cut(df[col], 10, labels=np.arange(10))
    
df['fold'] = 0
mskf = MultilabelStratifiedKFold(n_splits=5, shuffle=True, random_state=0)
for fold, (train_index, test_index) in enumerate(mskf.split(df, df[['img_id']+strf_cols])):
    df.loc[test_index, 'fold'] = fold

/home/trytolose/miniconda3/envs/audio/lib/python3.8/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass shuffle=True, random_state=0 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


In [108]:
df[df['fold']==5]['img_id'].value_counts()

Series([], Name: img_id, dtype: int64)

In [28]:
import math
math.ceil(3.3)

4

In [88]:
df

,img_id,glomerulus_pix,medulla,cortex,outer_stripe,Inner medulla,Outer Medulla,file,fold
0,2f6ecfcdf,0,6,0,0,0,0,/home/trytolose/rinat/kaggle/hubmap/input/trai...,0
1,2f6ecfcdf,0,2,0,0,0,0,/home/trytolose/rinat/kaggle/hubmap/input/trai...,4
2,2f6ecfcdf,0,8,0,0,0,0,/home/trytolose/rinat/kaggle/hubmap/input/trai...,4
3,2f6ecfcdf,0,9,0,0,0,0,/home/trytolose/rinat/kaggle/hubmap/input/trai...,0
4,2f6ecfcdf,0,8,0,0,0,0,/home/trytolose/rinat/kaggle/hubmap/input/trai...,4
...,...,...,...,...,...,...,...,...,...
9601,1e2425f28,0,0,4,0,0,0,/home/trytolose/rinat/kaggle/hubmap/input/trai...,1
9602,1e2425f28,0,0,5,0,0,0,/home/trytolose/rinat/kaggle/hubmap/input/trai...,2
9603,1e2425f28,0,0,7,0,0,0,/home/trytolose/rinat/kaggle/hubmap/input/trai...,4
9604,1e2425f28,0,0,7,0,0,0,/home/trytolose/rinat/kaggle/hubmap/input/trai...,0
